<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#DSE-203:-Final-Project" data-toc-modified-id="DSE-203:-Final-Project-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>DSE 203: Final Project</a></span><ul class="toc-item"><li><span><a href="#Author:-Adelle-Driker,-Bo-Yan,-Paul-Cabasag" data-toc-modified-id="Author:-Adelle-Driker,-Bo-Yan,-Paul-Cabasag-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Author: Adelle Driker, Bo Yan, Paul Cabasag</a></span></li><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Import Libraries</a></span></li><li><span><a href="#Connect-to-Neo4j-Database" data-toc-modified-id="Connect-to-Neo4j-Database-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Connect to Neo4j Database</a></span></li><li><span><a href="#Query-from-neo4j-and-create-data-frame-for-Recipe-List" data-toc-modified-id="Query-from-neo4j-and-create-data-frame-for-Recipe-List-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Query from neo4j and create data frame for Recipe List</a></span></li><li><span><a href="#Edit-data-for-lowercase" data-toc-modified-id="Edit-data-for-lowercase-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Edit data for lowercase</a></span></li><li><span><a href="#Perform-extraction-of-nouns-from-summary" data-toc-modified-id="Perform-extraction-of-nouns-from-summary-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Perform extraction of nouns from summary</a></span></li><li><span><a href="#Load-in-foodon-data" data-toc-modified-id="Load-in-foodon-data-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Load in foodon data</a></span></li><li><span><a href="#Perform-exact-valuematching-between-nouns-and-ingredients" data-toc-modified-id="Perform-exact-valuematching-between-nouns-and-ingredients-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Perform exact valuematching between nouns and ingredients</a></span></li><li><span><a href="#Create-dictionary-to-store-all-matched-ingredients-items" data-toc-modified-id="Create-dictionary-to-store-all-matched-ingredients-items-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Create dictionary to store all matched ingredients items</a></span></li><li><span><a href="#Transfer-ingredients-matches-into-neo4j" data-toc-modified-id="Transfer-ingredients-matches-into-neo4j-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Transfer ingredients matches into neo4j</a></span></li><li><span><a href="#Extract-calorie,-fat,-and-protein-values" data-toc-modified-id="Extract-calorie,-fat,-and-protein-values-1.11"><span class="toc-item-num">1.11&nbsp;&nbsp;</span>Extract calorie, fat, and protein values</a></span></li><li><span><a href="#Transfer-calorie,-fat,-and-protein-into-neo4j" data-toc-modified-id="Transfer-calorie,-fat,-and-protein-into-neo4j-1.12"><span class="toc-item-num">1.12&nbsp;&nbsp;</span>Transfer calorie, fat, and protein into neo4j</a></span></li></ul></li></ul></div>

# DSE 203: Final Project
## Author: Adelle Driker, Bo Yan, Paul Cabasag

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from py2neo import Graph, Node, Relationship, NodeMatcher
import pandas as pd
import numpy as np 
from sklearn.metrics import precision_score, recall_score
from collections import Counter
import nltk
import re

## Connect to Neo4j Database

In [14]:
url="bolt://localhost:7687"
pwd="12345"
graph=Graph(url, auth=('neo4j', pwd))

## Query from neo4j and create data frame for Recipe List

In [15]:
query = """
        MATCH (n:Recipe) RETURN n.id, n.title, n.summary, n.readyInMinutes, n.pricePerServing, n.glutenFree, n.dairyFree, n.vegan
"""
df_recipelist=graph.query(query).to_data_frame()

#clean data from Recipe List
df_recipelist=df_recipelist.dropna()

#check original data
df_recipelist

,n.id,n.title,n.summary,n.readyInMinutes,n.pricePerServing,n.glutenFree,n.dairyFree,n.vegan
0,636385.0,Buckwheat Banana Pancakes,The recipe Buckwheat Banana Pancakes can be ma...,30.0,134.63,True,False,False
1,637832.0,Chia Seed Pudding,"You can never have too many dessert recipes, s...",45.0,154.30,True,True,False
2,633078.0,Authentic Italian Biscotti,Authentic Italian Biscotti might be just the <...,45.0,9.73,False,True,False
3,632155.0,Almond Nut Butter,"Almond Nut Butter is a <b>gluten free, dairy f...",45.0,43.63,True,True,True
4,642178.0,Egg and Dairy Free Pancakes,If you have around <b>about 45 minutes</b> to ...,45.0,7.50,False,True,False
...,...,...,...,...,...,...,...,...
928,636393.0,Buckwheat Galette With An Egg,"You can never have too many side dish recipes,...",45.0,17.73,False,False,False
929,639597.0,Classic French Onion Soup,If you want to add more <b>Mediterranean</b> r...,45.0,293.89,False,False,False
930,633160.0,"Avocado Toast With Caramelized Sweet Onion, Gr...","Avocado Toast With Caramelized Sweet Onion, Gr...",45.0,247.74,False,True,True
931,652284.0,Molten Chocolate Liquor Cakes,Molten Chocolate Liquor Cakes might be just th...,45.0,94.52,True,False,False


## Edit data for lowercase

In [16]:
df_recipelist=df_recipelist.groupby('n.summary').head(912)
df_recipelist=df_recipelist.reset_index(drop=True)
df_recipelist['n.summary']=df_recipelist['n.summary'].str.lower()

## Perform extraction of nouns from summary

In [17]:
%%time

is_noun=lambda pos: pos[:2]=='NN'

nouns=[]
stop_words=['get', 'time', 'you', 'me', 'i', 'we', 'might', 'be', 'just', 'are', 'is', 'for', 'this', 'it','<b>', '/', '>', '<', 'minutes', 'calories', 'recipe', 'figure']

for i in range(df_recipelist.shape[0]):
    tokenized=nltk.word_tokenize(df_recipelist['n.summary'][i])
    nouns_text=[word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos) and word not in stop_words]
    nouns.append(nouns_text)
    
len(nouns)    

Wall time: 15.3 s


911

## Load in foodon data

In [18]:
ingredients=pd.read_csv('ingredients.csv')
ingredients

,Ingredients
0,all purpose flour
1,almond
2,apple cider vinegar
3,bacon
4,baker's yeast
...,...
137,taco
138,arugula
139,yogurt
140,cherry


## Perform exact valuematching between nouns and ingredients

In [19]:
%%time

matches=[]

for i in range(len(nouns)):
    w_matches=[]
    for j in range(ingredients.shape[0]):
        if ingredients['Ingredients'][j] in nouns[i]:
            w_matches.append(ingredients['Ingredients'][j])
    matches.append(w_matches)

Wall time: 960 ms


In [20]:
#export results
df_recipelist.to_csv('ingredientsVM.csv')

#output results
df_recipelist['matches']=matches
df_recipelist

,n.id,n.title,n.summary,n.readyInMinutes,n.pricePerServing,n.glutenFree,n.dairyFree,n.vegan,matches
0,636385.0,Buckwheat Banana Pancakes,the recipe buckwheat banana pancakes can be ma...,30.0,134.63,True,False,False,"[banana, buckwheat, cinnamon, quinoa]"
1,637832.0,Chia Seed Pudding,"you can never have too many dessert recipes, s...",45.0,154.30,True,True,False,"[almond, milk]"
2,633078.0,Authentic Italian Biscotti,authentic italian biscotti might be just the <...,45.0,9.73,False,True,False,"[almond, biscotti]"
3,632155.0,Almond Nut Butter,"almond nut butter is a <b>gluten free, dairy f...",45.0,43.63,True,True,True,"[almond, butter, chocolate]"
4,642178.0,Egg and Dairy Free Pancakes,if you have around <b>about 45 minutes</b> to ...,45.0,7.50,False,True,False,"[egg, ranch]"
...,...,...,...,...,...,...,...,...,...
906,636393.0,Buckwheat Galette With An Egg,"you can never have too many side dish recipes,...",45.0,17.73,False,False,False,"[egg, salt, spinach, buckwheat]"
907,639597.0,Classic French Onion Soup,if you want to add more <b>mediterranean</b> r...,45.0,293.89,False,False,False,"[onion, parsley]"
908,633160.0,"Avocado Toast With Caramelized Sweet Onion, Gr...","avocado toast with caramelized sweet onion, gr...",45.0,247.74,False,True,True,"[basil, garlic, onion, salt, avocado, grape]"
909,652284.0,Molten Chocolate Liquor Cakes,molten chocolate liquor cakes might be just th...,45.0,94.52,True,False,False,[chocolate]


## Create dictionary to store all matched ingredients items

In [21]:
%%time

match_dict = {}

for i in range(df_recipelist.shape[0]):
    curr_id = df_recipelist['n.title'][i]
    if curr_id in match_dict:
        match_dict[curr_id].append(df_recipelist['matches'][i])
    else:
        match_dict[curr_id] = df_recipelist['matches'][i]

match_dict

Wall time: 11 ms


{'Buckwheat Banana Pancakes': ['banana', 'buckwheat', 'cinnamon', 'quinoa'],
 'Chia Seed Pudding': ['almond', 'milk'],
 'Authentic Italian Biscotti': ['almond', 'biscotti'],
 'Almond Nut Butter': ['almond', 'butter', 'chocolate'],
 'Egg and Dairy Free Pancakes': ['egg', 'ranch'],
 'Vegan Strawberry Shortcake served with Vegan Whipped Cream': ['almond',
  'milk',
  'cream'],
 'Cake with lemon, rosewater and pistachios': ['lemon', 'pomegranate'],
 'Barbecue Chicken Cauliflower Couscous Bowls': ['chicken',
  'salt',
  'cauliflower'],
 'Blueberry, Chocolate & Cocao Superfood Pancakes - Gluten-Free/Paleo/Vegan': ['almond',
  'blueberry',
  'butter',
  'chocolate',
  'buckwheat',
  'blackberry'],
 'Amaretto Cheesecake': ['margarine', 'cream'],
 'Gluten Free Almond Blueberry Coffee Cake': ['almond',
  'blueberry',
  'berry',
  'coffee',
  'yogurt'],
 'Almond Joy Protein Shake': ['almond', 'coconut', 'milk', 'chocolate'],
 'Banana Milo Milkshake': ['banana', 'strawberry', 'cream'],
 'Almond Ho

## Transfer ingredients matches into neo4j

In [24]:
matcher = NodeMatcher(graph)

for key in match_dict:
    curr_node = matcher.match("Recipe", title = key).first()
    curr_node.setdefault("ingredient_matches", "")
    graph.push(curr_node)
    
for key in match_dict:
    if '"' in key:
        graph.run("MATCH (r:Recipe {title: '%s'}) SET r.ingredient_matches = $im" % str(key), im = str(match_dict[key]))
    else:
        graph.run('MATCH (r:Recipe {title: "%s"}) SET r.ingredient_matches = $im' % str(key), im = str(match_dict[key])) 

## Extract calorie, fat, and protein values

In [25]:
#functions
def find_all_places(text,word):
    word_places = []
    i=0
    while True:
        word_place = text.find(word,i)
        i+=len(word)+word_place
        if i>=len(text):
            break
        if word_place<0:
            break
        word_places.append(word_place)
    return word_places
def find_all_combination(text,start,end):
    start_places = find_all_places(text,start)
    end_places = find_all_places(text,end)
    combination_list = []
    for start_place in start_places:
        for end_place in end_places:
            print(start_place)
            print(end_place)
            if start_place>=end_place:
                continue
            combination_list.append(text[start_place:end_place])
    return combination_list

#calories per recipe
calories=[]
for i in df_recipelist['n.summary']:
    calories.append(re.findall(r'\d+', find_all_combination(i,'<b>','calories</b>')[0])[-1])
print(calories)

#fats per recipe
fats=[]
for i in df_recipelist['n.summary']:
    fats.append(re.findall(r'\d+', find_all_combination(i,'<b>','fat</b>')[1])[-1])
print(fats)

#protiens per recipe
proteins=[]
for i in df_recipelist['n.summary']:
    proteins.append(re.findall(r'\d+', find_all_combination(i,'<b>','protein</b>')[0])[-1])
print(proteins)   

49
158
151
158
234
158
143
150
164
150
535
150
45
119
113
119
193
119
400
119
23
146
139
146
186
146
622
146
19
313
118
313
178
313
327
313
73
150
143
150
440
150
47
112
105
112
490
112
131
138
152
138
636
138
24
318
229
318
311
318
587
318
21
29
43
29
71
29
268
29
64
211
148
211
229
211
638
211
158
165
179
165
606
165
87
237
125
237
230
237
557
237
70
209
109
209
201
209
639
209
26
215
62
215
103
215
208
215
477
215
4
144
45
144
137
144
246
144
523
144
36
147
140
147
187
147
451
147
105
242
143
242
256
242
544
242
158
212
205
212
715
212
54
253
102
253
247
253
567
253
4
155
42
155
148
155
416
155
125
132
146
132
550
132
7
151
144
151
165
151
508
151
115
254
156
254
294
254
15
83
76
83
97
83
498
83
151
158
172
158
563
158
97
104
118
104
360
104
92
178
171
178
293
178
24
230
166
230
223
230
462
230
122
129
143
129
727
129
121
128
142
128
441
128
147
309
188
309
349
309
721
309
74
81
95
81
365
81
101
109
123
109
589
109
7
146
139
146
160
146
342
146
99
236
137
236
250
236
676
236
130
137

In [26]:
#adding calories and fats as columns to recipe dataframe
df_recipelist['calories']=calories
df_recipelist['fats']=fats
df_recipelist['proteins']=proteins

In [27]:
%%time
#create dictionary for calories
match_dict_calories = {}

for i in range(df_recipelist.shape[0]):
    curr_id = df_recipelist['n.title'][i]
    if curr_id in match_dict_calories:
        match_dict_calories[curr_id].append(df_recipelist['calories'][i])
    else:
        match_dict_calories[curr_id] = df_recipelist['calories'][i]
        
match_dict_calories

Wall time: 15 ms


{'Buckwheat Banana Pancakes': '455',
 'Chia Seed Pudding': '142',
 'Authentic Italian Biscotti': '54',
 'Almond Nut Butter': '118',
 'Egg and Dairy Free Pancakes': '68',
 'Vegan Strawberry Shortcake served with Vegan Whipped Cream': '249',
 'Cake with lemon, rosewater and pistachios': '333',
 'Barbecue Chicken Cauliflower Couscous Bowls': '440',
 'Blueberry, Chocolate & Cocao Superfood Pancakes - Gluten-Free/Paleo/Vegan': '559',
 'Amaretto Cheesecake': '7616',
 'Gluten Free Almond Blueberry Coffee Cake': '298',
 'Almond Joy Protein Shake': '485',
 'Banana Milo Milkshake': '268',
 'Almond Horns': '7267',
 'Cranberry Crumb Bars': '163',
 '5 Minute Gluten Free Wonder Buns': '258',
 'Chocolate Chip Coconut Muffins': '274',
 'Snap Pea and Green Bean Salad with Arugula Pesto': '441',
 'Salmon Caesar Salad': '393',
 'Bleu Cheese Stuffed Wine Cherries With Honeycomb': '88',
 'Caramel Peanut Fudge Cake': '489',
 'Raspberry Walnut Coffee Cake': '189',
 'Cherry-Berry Pie With Agave Nectar All-But

In [28]:
%%time
#create dictionary for fats
match_dict_fats = {}

for i in range(df_recipelist.shape[0]):
    curr_id = df_recipelist['n.title'][i]
    if curr_id in match_dict_fats:
        match_dict_fats[curr_id].append(df_recipelist['fats'][i])
    else:
        match_dict_fats[curr_id] = df_recipelist['fats'][i]
        
match_dict_fats

Wall time: 12 ms


{'Buckwheat Banana Pancakes': '15',
 'Chia Seed Pudding': '6',
 'Authentic Italian Biscotti': '2',
 'Almond Nut Butter': '10',
 'Egg and Dairy Free Pancakes': '3',
 'Vegan Strawberry Shortcake served with Vegan Whipped Cream': '14',
 'Cake with lemon, rosewater and pistachios': '19',
 'Barbecue Chicken Cauliflower Couscous Bowls': '25',
 'Blueberry, Chocolate & Cocao Superfood Pancakes - Gluten-Free/Paleo/Vegan': '21',
 'Amaretto Cheesecake': '410',
 'Gluten Free Almond Blueberry Coffee Cake': '11',
 'Almond Joy Protein Shake': '41',
 'Banana Milo Milkshake': '9',
 'Almond Horns': '526',
 'Cranberry Crumb Bars': '9',
 '5 Minute Gluten Free Wonder Buns': '15',
 'Chocolate Chip Coconut Muffins': '11',
 'Snap Pea and Green Bean Salad with Arugula Pesto': '36',
 'Salmon Caesar Salad': '29',
 'Bleu Cheese Stuffed Wine Cherries With Honeycomb': '1',
 'Caramel Peanut Fudge Cake': '38',
 'Raspberry Walnut Coffee Cake': '7',
 'Cherry-Berry Pie With Agave Nectar All-Butter Crust': '22',
 'Decade

In [29]:
%%time
#create dictionary for proteins
match_dict_proteins = {}

for i in range(df_recipelist.shape[0]):
    curr_id = df_recipelist['n.title'][i]
    if curr_id in match_dict_proteins:
        match_dict_proteins[curr_id].append(df_recipelist['proteins'][i])
    else:
        match_dict_proteins[curr_id] = df_recipelist['proteins'][i]
        
match_dict_proteins

Wall time: 8.98 ms


{'Buckwheat Banana Pancakes': '11',
 'Chia Seed Pudding': '4',
 'Authentic Italian Biscotti': '1',
 'Almond Nut Butter': '4',
 'Egg and Dairy Free Pancakes': '1',
 'Vegan Strawberry Shortcake served with Vegan Whipped Cream': '4',
 'Cake with lemon, rosewater and pistachios': '6',
 'Barbecue Chicken Cauliflower Couscous Bowls': '23',
 'Blueberry, Chocolate & Cocao Superfood Pancakes - Gluten-Free/Paleo/Vegan': '12',
 'Amaretto Cheesecake': '113',
 'Gluten Free Almond Blueberry Coffee Cake': '6',
 'Almond Joy Protein Shake': '24',
 'Banana Milo Milkshake': '4',
 'Almond Horns': '234',
 'Cranberry Crumb Bars': '2',
 '5 Minute Gluten Free Wonder Buns': '11',
 'Chocolate Chip Coconut Muffins': '5',
 'Snap Pea and Green Bean Salad with Arugula Pesto': '12',
 'Salmon Caesar Salad': '27',
 'Bleu Cheese Stuffed Wine Cherries With Honeycomb': '1',
 'Caramel Peanut Fudge Cake': '17',
 'Raspberry Walnut Coffee Cake': '2',
 'Cherry-Berry Pie With Agave Nectar All-Butter Crust': '5',
 'Decadent Bla

## Transfer calorie, fat, and protein into neo4j

In [30]:
#transfer calories into neo4j
matcher = NodeMatcher(graph)

for key in match_dict_calories:
    curr_node = matcher.match("Recipe", title = key).first()
    curr_node.setdefault("calories", "")
    curr_node.calories=match_dict_calories[key]
    graph.push(curr_node)
    
for key in match_dict_calories:
    if '"' in key:
        graph.run("MATCH (r:Recipe {title: '%s'}) SET r.calories = $cal" % str(key), cal = str(match_dict_calories[key]))
    else:
        graph.run('MATCH (r:Recipe {title: "%s"}) SET r.calories = $cal' % str(key), cal = str(match_dict_calories[key]))    

In [31]:
#transfer fats into neo4j
matcher = NodeMatcher(graph)

for key in match_dict_fats:
    curr_node = matcher.match("Recipe", title = key).first()
    curr_node.setdefault("fats", "")
    curr_node.fats=match_dict_fats[key]
    graph.push(curr_node)
    
for key in match_dict_fats:
    if '"' in key:
        graph.run("MATCH (r:Recipe {title: '%s'}) SET r.fats = $fat" % str(key), fat = str(match_dict_fats[key]))
    else:
        graph.run('MATCH (r:Recipe {title: "%s"}) SET r.fats = $fat' % str(key), fat = str(match_dict_fats[key]))    

In [32]:
#transfer proteins into neo4j
matcher = NodeMatcher(graph)

for key in match_dict_proteins:
    curr_node = matcher.match("Recipe", title = key).first()
    curr_node.setdefault("proteins", "")
    curr_node.proteins=match_dict_proteins[key]
    graph.push(curr_node)
    
for key in match_dict_proteins:
    if '"' in key:
        graph.run("MATCH (r:Recipe {title: '%s'}) SET r.proteins = $pro" % str(key), pro = str(match_dict_proteins[key]))
    else:
        graph.run('MATCH (r:Recipe {title: "%s"}) SET r.proteins = $pro' % str(key), pro = str(match_dict_proteins[key]))    